In [12]:
import pandas as pd
import numpy as np
import threading
import time

In [13]:
class MyThread(threading.Thread):
    def __init__(self, target=None, args=(), **kwargs):
        super(MyThread, self).__init__()
        self._target = target
        self._args = args
        self._kwargs = kwargs

    def run(self):
        if self._target == None:
            return
        self.__result__ = self._target(*self._args, **self._kwargs)

    def get_result(self):
        self.join()#當需要取得結果值的時候阻塞等待子執行緒完成
        return self.__result__

In [3]:
df = pd.read_csv('/Users/uekilee/Desktop/train.csv')
df.head()

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,113261,38038,513.80,5,0,N,0,N,N,...,0,33,172652.0,457,59333,N,0,102,0,516056
1,0,134508,45725,465.62,5,0,N,2,N,N,...,0,9,105114.0,451,0,N,5817,102,0,4376
2,6881,15408,188328,513.80,5,0,N,0,N,N,...,0,6,152458.0,457,59333,N,0,102,0,483434
3,6716,157159,29967,1016.11,5,62,N,5,N,N,...,0,5,172946.0,247,50436,N,3281,102,0,1407164
4,5975,105985,81305,713.66,5,62,N,4,N,N,...,0,6,182129.0,263,93775,N,5817,102,0,1051004


In [122]:
def createCol(df,s,e):
    start = time.time()
    print(s,e,'start')
    columns = df.columns
    df_new = pd.DataFrame(columns=columns)
    df_new['3d'] = None
    df_gb = df.groupby('cano')
    for i in df['cano'].unique()[s:e]:
        df_sort = df_gb.get_group(i)
        df_sort['3d'] = df_sort['locdt'].apply(lambda x : df_sort[(df_sort['locdt']<x) & (df_sort['locdt']>x-3)].shape[0])

        df_new = pd.concat([df_new, df_sort], sort=False, ignore_index=False)
        
#     df_new.to_csv(f'data/test{s}_{e}.csv',index=True)
    print(s,e,'finish',time.time()-start)
    return df_new

In [5]:
len(df['cano'].unique())

129413

In [15]:
#間隔2000為65組
blocks = []
for i in range(0,129413,2000):
    blocks.append([i,i+2000])
# blocks

In [16]:
threads = []
# count = 0
for i in blocks:
    threads.append(MyThread(target=createCol , args=( df , i[0] , i[1]) ))
for i in threads:
    i.start()
#避免一次開太多執行緒 .join()為該執行緒結束才會繼續主程式
#     if count%50==49:
#         i.join()
#     count+=1
# print('完成')


0 2000 start
2000 4000 start
4000 60006000 start 
8000 10000 8000start
 10000start
 12000 14000 start
12000 start
14000 16000 start
16000 18000 start
18000 20000 start
20000 2200022000 start
 24000 start
24000 26000 start
26000 2800028000  start
3000030000 start
32000 32000 start
 34000 start
34000 36000 start
36000 38000 start
38000 40000 start
40000 42000 start
42000 44000 start
44000 46000 start
46000 48000 start
48000 50000 start
50000 52000 start
52000 54000 start
54000 56000 start
56000 58000 start
58000 60000 start
60000 62000 start
62000 64000 start
64000 66000 start
66000 68000 start
68000 70000 start
70000 72000 start
72000 74000 start
74000 76000 start
76000 78000 start
78000 80000 start
80000 82000 start
82000 84000 start
84000 86000 start
86000 88000 start
88000 90000 start
90000 92000 start
92000 94000 start
94000 96000 start
96000 98000 start
98000 100000 start
100000 102000 start
102000 104000 start
104000 106000 start
106000 108000 start
108000 110000 start
110000 1120

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [29]:
#確認執行緒是否繼續運作 True為運作中
index = 0
for i in threads:
    print(index, i.isAlive())
    index+=1

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
21 False
22 False
23 False
24 False
25 False
26 False
27 False
28 False
29 False
30 False
31 False
32 False
33 False
34 False
35 False
36 False
37 False
38 False
39 False
40 False
41 False
42 False
43 False
44 False
45 False
46 False
47 False
48 False
49 False
50 False
51 False
52 False
53 False
54 False
55 False
56 False
57 False
58 False
59 False
60 False
61 False
62 False
63 False
64 False


In [30]:
#待多執行緒皆停止，才可合併df
df_final = threads[0].get_result()
for i in threads[1:]:
    df_final = pd.concat([df_final, i.get_result()], sort=False, ignore_index=False)

df_final.shape

(1521787, 24)

In [35]:
df.shape

(1521787, 23)

In [36]:
df_final.sort_index().to_csv(f'data/train_new.csv',index=False)

# 以下測試用

In [ ]:
#單隻程式測試
%%time
createCol(df,0,100)

In [ ]:
#小量多執行緒測試
threads = []
for i in [[0,10],[10,20]]:
    threads.append(threading.Thread(target=createCol , args=( df , i[0] , i[1]) ))
for i in threads:
    i.start()

In [11]:
#可取出多執行緒回傳結果 (回傳為df)
for i in threads:
    print(i.get_result().shape)

(5239, 24)
(4385, 24)
(2655, 24)
(3411, 24)
(2728, 24)
(2857, 24)
(2699, 24)
(2695, 24)
(2815, 24)
(3107, 24)


In [ ]:
#合併csv檔案
df_new = pd.DataFrame(columns=columns)
filepath = './data'
filename = os.listdir(f'{filepath}')
for filename in filenames:
    df_tmp = pd.read_csv(f'{filepath}/{filename}')
    df_new = pd.concat([df_new, df_tmp], sort=False, ignore_index=True)
df_new.to_csv(r'train_new.csv',index=False)

In [37]:
pd.read_csv('data/train_new.csv').head()

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey,3d
0,6881,113261,38038,513.80,5,0,N,0,N,N,...,33,172652.0,457,59333,N,0,102,0,516056,0
1,0,134508,45725,465.62,5,0,N,2,N,N,...,9,105114.0,451,0,N,5817,102,0,4376,1
2,6881,15408,188328,513.80,5,0,N,0,N,N,...,6,152458.0,457,59333,N,0,102,0,483434,0
3,6716,157159,29967,1016.11,5,62,N,5,N,N,...,5,172946.0,247,50436,N,3281,102,0,1407164,0
4,5975,105985,81305,713.66,5,62,N,4,N,N,...,6,182129.0,263,93775,N,5817,102,0,1051004,3


In [130]:
%%time
import multiprocessing
test = []
pool = multiprocessing.Pool() #process=
for i in [[0,2000],[2000,4000]]:
    test.append(pool.apply_async(createCol, (df,i[0],i[1])))
pool.close()
pool.join()

0 2000 start


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


2000 4000 start


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


2000 4000 finish 244.2263150215149
0 2000 finish 281.0106499195099
CPU times: user 2.3 s, sys: 2 s, total: 4.29 s
Wall time: 4min 46s


In [120]:
for i in test:
    try:
        i.successful()
    except:
        print('not yet')

In [115]:
for i in test:
    print(i.get().shape)

(5239, 24)
(4385, 24)


In [131]:
%%time
threads = []
# count = 0
for i in [[0,2000],[2000,4000]]:
    threads.append(MyThread(target=createCol , args=( df , i[0] , i[1]) ))
for i in threads:
    i.start()
for i in threads:
    i.join()

0 2000 start
2000 4000 start


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


2000 4000 finish 624.5388820171356
0 2000 finish 653.4977810382843
CPU times: user 8min 17s, sys: 40.5 s, total: 8min 58s
Wall time: 10min 53s


In [132]:
%%time
for i in [[0,2000],[2000,4000]]:
    createCol(df , i[0] , i[1])

0 2000 start


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


0 2000 finish 271.96820616722107
2000 4000 start
2000 4000 finish 246.77330207824707
CPU times: user 8min 8s, sys: 34.3 s, total: 8min 42s
Wall time: 8min 38s
